<a href="https://colab.research.google.com/github/RonanD10/Tennis-Match-Prediction/blob/main/data-processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### Project steps ###
# 0) Understand dataset: missing values, datatypes, make new columns, etc
# 1) Create player class with 'next_elo', 'time_without_play', etc, attributes
# 2) Write data analysis code to run through the dataset
# 3) Display results and comment

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
 !git clone https://github.com/JeffSackmann/tennis_atp.git

fatal: destination path 'tennis_atp' already exists and is not an empty directory.


In [ ]:
### Set up dataframes

all_players = pd.read_csv('tennis_atp/atp_players.csv')
all_players['name'] = all_players['name_first'] + ' ' + all_players['name_last']
all_players.drop(['name_first', 'name_last'], axis=1, inplace=True)

# Concatenate all years
all_matches = pd.read_csv('tennis_atp/atp_matches_1968.csv')

for i in range(1969, 2025):
    df = pd.read_csv(f'tennis_atp/atp_matches_{i}.csv')
    all_matches = pd.concat([all_matches, df])

In [ ]:
all_matches.shape, all_players.shape

((193337, 49), (65019, 7))

In [ ]:
all_players.head()

,player_id,hand,dob,ioc,height,wikidata_id,name
0,100001,R,19131122.0,USA,185.0,Q54544,Gardnar Mulloy
1,100002,R,19210620.0,ECU,168.0,Q54581,Pancho Segura
2,100003,R,19271002.0,AUS,180.0,Q962049,Frank Sedgman
3,100004,R,19271011.0,ITA,NaN,Q1258752,Giuseppe Merlo
4,100005,R,19280509.0,USA,188.0,Q53554,Richard Gonzalez


In [ ]:
all_matches.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,1968-2029,Dublin,Grass,32.0,A,19680708,270,112411,NaN,NaN,Doug Smith,U,NaN,AUS,NaN,110196,NaN,NaN,Peter Ledbetter,U,NaN,IRL,24.0,6-1 7-5,3,R32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1968-2029,Dublin,Grass,32.0,A,19680708,271,126914,NaN,NaN,Louis Pretorius,R,NaN,RSA,NaN,209536,NaN,NaN,Maurice Pollock,U,NaN,IRL,NaN,6-1 6-1,3,R32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1968-2029,Dublin,Grass,32.0,A,19680708,272,209523,NaN,NaN,Cecil Pedlow,U,NaN,IRL,NaN,209535,NaN,NaN,John Mulvey,U,NaN,IRL,NaN,6-2 6-2,3,R32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1968-2029,Dublin,Grass,32.0,A,19680708,273,100084,NaN,NaN,Tom Okker,R,178.0,NED,24.3,209534,NaN,NaN,Unknown Fearmon,U,NaN,NaN,NaN,6-1 6-1,3,R32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1968-2029,Dublin,Grass,32.0,A,19680708,274,100132,NaN,NaN,Armistead Neely,R,NaN,USA,21.3,209533,NaN,NaN,Harry Sheridan,U,NaN,IRL,NaN,6-2 6-4,3,R32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
all_matches.columns

Index(['tourney_id', 'tourney_name', 'surface', 'draw_size', 'tourney_level',
       'tourney_date', 'match_num', 'winner_id', 'winner_seed', 'winner_entry',
       'winner_name', 'winner_hand', 'winner_ht', 'winner_ioc', 'winner_age',
       'loser_id', 'loser_seed', 'loser_entry', 'loser_name', 'loser_hand',
       'loser_ht', 'loser_ioc', 'loser_age', 'score', 'best_of', 'round',
       'minutes', 'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon',
       'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df', 'l_svpt',
       'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced',
       'winner_rank', 'winner_rank_points', 'loser_rank', 'loser_rank_points'],
      dtype='object')

In [ ]:
# Add columns to track player *pre-match* Elo

# Surface Elo
all_matches['winner_hard_elo_before'] = 1500.0
all_matches['loser_hard_elo_before'] = 1500.0
all_matches['winner_hard_elo_after'] = 1500.0
all_matches['loser_hard_elo_after'] = 1500.0

all_matches['winner_clay_elo_before'] = 1500.0
all_matches['loser_clay_elo_before'] = 1500.0
all_matches['winner_clay_elo_after'] = 1500.0
all_matches['loser_clay_elo_after'] = 1500.0

all_matches['winner_grass_elo_before'] = 1500.0
all_matches['loser_grass_elo_before'] = 1500.0
all_matches['winner_grass_elo_after'] = 1500.0
all_matches['loser_grass_elo_after'] = 1500.0

all_matches['winner_carpet_elo_before'] = 1500.0
all_matches['loser_carpet_elo_before'] = 1500.0
all_matches['winner_carpet_elo_after'] = 1500.0
all_matches['loser_carpet_elo_after'] = 1500.0

# Overall Elo
all_matches['winner_elo_before'] = 1500.0
all_matches['loser_elo_before'] = 1500.0
all_matches['winner_elo_after'] = 1500.0
all_matches['loser_elo_after'] = 1500.0

# Initialise Elo calculation tracking columns
all_players['hard_elo'] = 1500.0
all_players['clay_elo'] = 1500.0
all_players['grass_elo'] = 1500.0
all_players['carpet_elo'] = 1500.0
all_players['overall_elo'] = 1500.0

Elo formula for a player $a$:

$$R’_a = R_a + K(S_a — E_a),$$

- $R_a$ is current Elo
- $R'_a$ is the new Elo score
- $E_a$ is the expected outcome versus player $b$, given by $$E_a = \frac{Q_a}{Q_a + Q_b},$$ with $Q_a = 10^{\frac{R_a}{c}}, Q_b = 10^{\frac{R_b}{c}}$. Note that $0 ≤ E_a ≤ 1$.
- $S_a$ is the actual outcome, with $S_a = 0$ or $1$ for loss and win respectively.
- $K$ and $c$ are free scaling parameters, with values $K = 32$ and $c = 400$, as is standard.


In [ ]:
# Elo calculator
def elo(R_a, R_b):
    """
    winner = player a
    loser = player b

    Input: Elo R_a of player a, Elo R_b of player b
    Output: New respective Elos R_a_new ane R_b_new
    """
    Q_a = 10 ** (R_a / 400)
    Q_b = 10 ** (R_b / 400)

    E_a = Q_a / (Q_a + Q_b)
    E_b = Q_b / (Q_b + Q_a)

    S_a = 1
    S_b = 0

    R_a_new = R_a + 32 * (S_a - E_a)
    R_b_new = R_b + 32 * (S_b - E_b)

    return np.round(R_a_new, 3), np.round(R_b_new, 3)

In [ ]:
all_matches

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,winner_hard_elo_before,loser_hard_elo_before,winner_hard_elo_after,loser_hard_elo_after,winner_clay_elo_before,loser_clay_elo_before,winner_clay_elo_after,loser_clay_elo_after,winner_grass_elo_before,loser_grass_elo_before,winner_grass_elo_after,loser_grass_elo_after,winner_carpet_elo_before,loser_carpet_elo_before,winner_carpet_elo_after,loser_carpet_elo_after,winner_elo_before,loser_elo_before,winner_elo_after,loser_elo_after
0,1968-2029,Dublin,grass,32.0,A,19680708,270,112411,NaN,NaN,Doug Smith,U,NaN,AUS,NaN,110196,NaN,NaN,Peter Ledbetter,U,NaN,IRL,24.0,6-1 7-5,3,R32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0
1,1968-2029,Dublin,grass,32.0,A,19680708,271,126914,NaN,NaN,Louis Pretorius,R,NaN,RSA,NaN,209536,NaN,NaN,Maurice Pollock,U,NaN,IRL,NaN,6-1 6-1,3,R32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0
2,1968-2029,Dublin,grass,32.0,A,19680708,272,209523,NaN,NaN,Cecil Pedlow,U,NaN,IRL,NaN,209535,NaN,NaN,John Mulvey,U,NaN,IRL,NaN,6-2 6-2,3,R32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0
3,1968-2029,Dublin,grass,32.0,A,19680708,273,100084,NaN,NaN,Tom Okker,R,178.0,NED,24.3,209534,NaN,NaN,Unknown Fearmon,U,NaN,NaN,NaN,6-1 6-1,3,R32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0
4,1968-2029,Dublin,grass,32.0,A,19680708,274,100132,NaN,NaN,Armistead Neely,R,NaN,USA,21.3,209533,NaN,NaN,Harry Sheridan,U,NaN,IRL,NaN,6-2 6-4,3,R32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1412,2024-M-DC-2024-WG2-PO-URU-MDA-01,Davis Cup WG2 PO: URU vs MDA,clay,4.0,D,20240203,5,212051,NaN,NaN,Joaquin Aguilar Cardozo,U,NaN,URU,18.8,209943,NaN,NaN,Ilya Snitari,U,NaN,MDA,21.8,6-1 6-0,3,RR,77.0,0.0,0.0,36.0,22.0,15.0,10.0,7.0,1.0,1.0,1.0,1.0,61.0,30.0,17.0,7.0,6.0,8.0,14.0,1109.0,8.0,740.0,34.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0
1413,2024-M-DC-2024-WG2-PO-VIE-RSA-01,Davis Cup WG2 PO: VIE vs RSA,hard,4.0,D,20240202,1,122533,NaN,NaN,Nam Hoang Ly,R,NaN,VIE,26.9,202475,NaN,NaN,Philip Henning,R,NaN,RSA,23.2,6-3 6-4,3,RR,131.0,1.0,1.0,86.0,60.0,40.0,13.0,10.0,8.0,9.0,2.0,1.0,56.0,41.0,25.0,6.0,9.0,1.0,4.0,554.0,67.0,748.0,32.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0
1414,2024-M-DC-2024-WG2-PO-

In [ ]:
# Sort by tournament and round
map = {'RR': 0, 'BR': 0, 'ER': 0 , 'R128': 1, 'R64': 2, 'R32': 3, 'R16': 4, 'QF': 5, 'SF': 6, 'F': 7}
all_matches['round'] = all_matches['round'].map(map)
all_matches.sort_values(by=['tourney_date', 'round'], inplace=True)
all_matches.reset_index(drop=False, inplace=True)

# Clean surface column
all_matches['surface'] = all_matches['surface'].str.lower()
all_matches['surface'] = all_matches['surface'].fillna('unknown')

In [ ]:
# Calculate Elos match by match

for i in range(0, len(all_matches)):

    winner_id = all_matches.iloc[i]['winner_id']
    loser_id = all_matches.iloc[i]['loser_id']
    surface = all_matches.iloc[i]['surface']

    # Elo before:
    all_matches.at[i, 'winner_elo_before'] = all_players.loc[all_players['player_id'] == winner_id, 'overall_elo'].iloc[0]
    all_matches.at[i, 'loser_elo_before'] = all_players.loc[all_players['player_id'] == loser_id, 'overall_elo'].iloc[0]

    all_matches.at[i, 'winner_hard_elo_before'] = all_players.loc[all_players['player_id'] == winner_id, 'hard_elo'].iloc[0]
    all_matches.at[i, 'loser_hard_elo_before'] = all_players.loc[all_players['player_id'] == loser_id, 'hard_elo'].iloc[0]

    all_matches.at[i, 'winner_grass_elo_before'] = all_players.loc[all_players['player_id'] == winner_id, 'grass_elo'].iloc[0]
    all_matches.at[i, 'loser_grass_elo_before'] = all_players.loc[all_players['player_id'] == loser_id, 'grass_elo'].iloc[0]

    all_matches.at[i, 'winner_clay_elo_before'] = all_players.loc[all_players['player_id'] == winner_id, 'clay_elo'].iloc[0]
    all_matches.at[i, 'loser_clay_elo_before'] = all_players.loc[all_players['player_id'] == loser_id, 'clay_elo'].iloc[0]

    all_matches.at[i, 'winner_carpet_elo_before'] = all_players.loc[all_players['player_id'] == winner_id, 'carpet_elo'].iloc[0]
    all_matches.at[i, 'loser_carpet_elo_before'] = all_players.loc[all_players['player_id'] == loser_id, 'carpet_elo'].iloc[0]

    # Elo after:
    # Overall Elo
    R_a = all_players.loc[all_players['player_id'] == winner_id, 'overall_elo'].iloc[0]
    R_b = all_players.loc[all_players['player_id'] == loser_id, 'overall_elo'].iloc[0]

    R_a_new, R_b_new = elo(R_a, R_b)

    all_matches.at[i, 'winner_elo_after'] = R_a_new
    all_matches.at[i, 'loser_elo_after'] = R_b_new
    all_players.loc[all_players['player_id'] == winner_id, 'overall_elo'] = R_a_new
    all_players.loc[all_players['player_id'] == loser_id, 'overall_elo'] = R_b_new

    # Surface Elo
    if surface != 'unknown':
        R_a = all_players.loc[all_players['player_id'] == winner_id, f'{surface}_elo'].iloc[0]
        R_b = all_players.loc[all_players['player_id'] == loser_id, f'{surface}_elo'].iloc[0]

        R_a_new, R_b_new = elo(R_a, R_b)

        all_matches.at[i, f'winner_{surface}_elo_after'] = R_a_new
        all_matches.at[i, f'loser_{surface}_elo_after'] = R_b_new
        all_players.loc[all_players['player_id'] == winner_id, f'{surface}_elo'] = R_a_new
        all_players.loc[all_players['player_id'] == loser_id, f'{surface}_elo'] = R_b_new

In [ ]:
all_matches

,index,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,winner_hard_elo_before,loser_hard_elo_before,winner_hard_elo_after,loser_hard_elo_after,winner_clay_elo_before,loser_clay_elo_before,winner_clay_elo_after,loser_clay_elo_after,winner_grass_elo_before,loser_grass_elo_before,winner_grass_elo_after,loser_grass_elo_after,winner_carpet_elo_before,loser_carpet_elo_before,winner_carpet_elo_after,loser_carpet_elo_after,winner_elo_before,loser_elo_before,winner_elo_after,loser_elo_after
0,3753,1968-T101,Bombay,clay,NaN,A,19671228,101,100092,NaN,NaN,Alex Metreveli,R,NaN,RUS,23.1,100113,NaN,NaN,Jan Kodes,R,175.0,CZE,21.8,6-4 3-6 6-1,3,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1500.000,1500.000,1500.0,1500.0,1500.000,1500.000,1516.000,1484.000,1500.000,1500.000,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.000,1500.000,1516.000,1484.000
1,3754,1968-T101,Bombay,clay,NaN,A,19671228,102,110991,NaN,NaN,Jaidip Mukerjea,R,NaN,IND,25.6,100070,NaN,NaN,Jan Kukal,R,NaN,CZE,25.2,5-7 10-8 6-4,3,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1500.000,1500.000,1500.0,1500.0,1500.000,1500.000,1516.000,1484.000,1500.000,1500.000,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.000,1500.000,1516.000,1484.000
2,3755,1968-T101,Bombay,clay,NaN,A,19671228,103,100051,NaN,NaN,Premjit Lall,R,NaN,IND,27.1,100119,NaN,NaN,Ilie Nastase,R,183.0,ROU,21.4,6-3 2-6 6-4,3,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1500.000,1500.000,1500.0,1500.0,1500.000,1500.000,1516.000,1484.000,1500.000,1500.000,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.000,1500.000,1516.000,1484.000
3,3756,1968-T101,Bombay,clay,NaN,A,19671228,104,100035,NaN,NaN,Ion Tiriac,R,185.0,ROU,28.6,114613,NaN,NaN,Shyam Minotra,U,NaN,IND,21.2,6-4 6-4,3,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1500.000,1500.000,1500.0,1500.0,1500.000,1500.000,1516.000,1484.000,1500.000,1500.000,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.000,1500.000,1516.000,1484.000
4,3760,1968-T102,Calcutta,unknown,NaN,A,19671228,101,100035,NaN,NaN,Ion Tiriac,R,185.0,ROU,28.6,100141,NaN,NaN,Zeljko Franulovic,R,NaN,CRO,20.5,6-3 6-3 4-6 8-6,5,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1500.000,1500.000,1500.0,1500.0,1516.000,1500.000,1500.000,1500.000,1500.000,1500.000,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1516.000,1500.000,1531.264,1484.736
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193332,1250,2024-0322,Geneva,clay,32.0,A,20240520,298,134770,2,NaN,Casper Ruud,R,183.0,NOR,25.4,207925,NaN,NaN,Flavio Cobolli,R,NaN,ITA,22.0,1-6 6-1 7-6(4),3,6,110.0,2.0,2.0,77.0,49.0,32.0,18.0,13.0,3.0,6.0,8.0,1.0,83.0,51.0,34.0,16.0,13.0,6.0,9.0,7.0,4185.0,56.0,860.0,1804.709,1566.592,1500.0,1500.0,1901.149,1590.551,1905.735,1585.965,1487.884,1500.000,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1926.081,1684.596,1932.462,1678.215
193333,1276,2024-7694,Lyon,clay,32.0,A,20240520,299,144869,6,NaN,Tomas Martin Etcheverry,R,196.0,ARG,24.8,209260,NaN,NaN,Luciano Darderi,R,NaN,ITA,22.2,6-2 6-4,3,6,90.0,8.0,1.0,52.0,30.0,25.0,14.0,9.0,3.0,3.0,1.0,2.0,53.0,28.0,17.0,14.0,9.0,3.0,6.0,29.0,1410.0,47.0,1002.0,1556.839,1482.164,1500.0,1500.0,1694.263

In [ ]:
# Export tables
all_matches.to_csv('all_matches.csv', index=False)
all_players.to_csv('all_players.csv', index=False)

In [ ]:
pd.read_csv('/content/tennis_atp/atp_rankings_70s.csv')

,ranking_date,rank,player,points
0,19730827,129,100005,NaN
1,19730827,114,100011,NaN
2,19730827,6,100016,NaN
3,19730827,19,100022,NaN
4,19730827,82,100025,NaN
...,...,...,...,...
20721,19771212,95,100586,NaN
20722,19771212,97,100363,NaN
20723,19771212,98,100176,NaN
20724,19771212,99,100233,NaN


In [ ]:
rk80 = pd.read_csv('/content/tennis_atp/atp_rankings_80s.csv')

In [ ]:
set(all_players['player_id']) == set(rk80['player'])

False

In [ ]:
all_rankings = pd.read_csv('/content/tennis_atp/atp_rankings_70s.csv')

for date in ['80s', '90s', '00s', '10s', '20s', 'current']:
    df = pd.read_csv(f'/content/tennis_atp/atp_rankings_{date}.csv')
    all_rankings = pd.concat([all_rankings, df])

In [ ]:
all_rankings.to_csv('all_rankings.csv')

In [ ]:
all_rankings['ranking_date'].max()

20240527

In [ ]:
all_matches['tourney_date'].max()

20240520

In [ ]:
all_rankings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Tennis Project/all_rankings.csv')

In [ ]:
all_matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193337 entries, 0 to 193336
Data columns (total 50 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   index               193337 non-null  int64  
 1   tourney_id          193337 non-null  object 
 2   tourney_name        193337 non-null  object 
 3   surface             190347 non-null  object 
 4   draw_size           192502 non-null  float64
 5   tourney_level       193337 non-null  object 
 6   tourney_date        193337 non-null  int64  
 7   match_num           193337 non-null  int64  
 8   winner_id           193337 non-null  int64  
 9   winner_seed         71544 non-null   object 
 10  winner_entry        16943 non-null   object 
 11  winner_name         193337 non-null  object 
 12  winner_hand         193327 non-null  object 
 13  winner_ht           176236 non-null  float64
 14  winner_ioc          193329 non-null  object 
 15  winner_age          192026 non-nul

In [ ]:
all_matches = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Tennis Project/all_matches.csv')

<ipython-input-3-1f2b3696e92f>:1: DtypeWarning: Columns (9,17) have mixed types. Specify dtype option on import or set low_memory=False.
  all_matches = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Tennis Project/all_matches.csv')


In [ ]:
all_matches[all_matches['tourney_date'] > 19900000].info()

<class 'pandas.core.frame.DataFrame'>
Index: 110397 entries, 82940 to 193336
Data columns (total 62 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   index               110397 non-null  int64  
 1   tourney_id          110397 non-null  object 
 2   tourney_name        110397 non-null  object 
 3   surface             110397 non-null  object 
 4   draw_size           110397 non-null  float64
 5   tourney_level       110397 non-null  object 
 6   tourney_date        110397 non-null  int64  
 7   match_num           110397 non-null  int64  
 8   winner_id           110397 non-null  int64  
 9   winner_seed         44673 non-null   object 
 10  winner_entry        13526 non-null   object 
 11  winner_name         110397 non-null  object 
 12  winner_hand         110390 non-null  object 
 13  winner_ht           107561 non-null  float64
 14  winner_ioc          110397 non-null  object 
 15  winner_age          110386 non-null

In [ ]:
for date in all_matches[all_matches['winner_rank'].isnull()]['tourney_date']:

    winner_id = all_matches.loc[all_matches['tourney_date'] == date, 'winner_id'].iloc[0]
    loser_id = all_matches.loc[all_matches['tourney_date'] == date, 'loser_id'].iloc[0]

    try:
        winner_rank = all_rankings.loc[(all_rankings['ranking_date'] == date) & (all_rankings['player'] == winner_id), 'rank'].iloc[0]
        loser_rank = all_rankings.loc[(all_rankings['ranking_date'] == date) & (all_rankings['player'] == loser_id), 'rank'].iloc[0]
        all_matches.loc[all_matches['tourney_date'] == date, 'winner_rank'] = winner_rank
        all_matches.loc[all_matches['tourney_date'] == date, 'loser_rank'] = loser_rank
    except:
        continue

In [ ]:
all_matches[(all_matches['winner_rank'].isnull()) & (all_matches['tourney_date'] > 20000000)]['tourney_level'].value_counts()

,count
tourney_level,
D,521
A,36
G,6
M,3


In [ ]:
all_matches.info()

In [ ]:
all_matches[all_matches['winner_rank'].isnull()]

,index,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,winner_elo,loser_elo,winner_hard_elo,loser_hard_elo,winner_clay_elo,loser_clay_elo,winner_grass_elo,loser_grass_elo,winner_carpet_elo,loser_carpet_elo,winner_overall_elo,loser_overall_elo
0,3753,1968-T101,Bombay,clay,NaN,A,19671228,101,100092,NaN,NaN,Alex Metreveli,R,NaN,RUS,23.1,100113,NaN,NaN,Jan Kodes,R,175.0,CZE,21.8,6-4 3-6 6-1,3,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.000,0.000,1516.000,1484.000,0.0,0.0,0.0,0.0,1516.000,1484.000
1,3754,1968-T101,Bombay,clay,NaN,A,19671228,102,110991,NaN,NaN,Jaidip Mukerjea,R,NaN,IND,25.6,100070,NaN,NaN,Jan Kukal,R,NaN,CZE,25.2,5-7 10-8 6-4,3,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.000,0.000,1516.000,1484.000,0.0,0.0,0.0,0.0,1516.000,1484.000
2,3755,1968-T101,Bombay,clay,NaN,A,19671228,103,100051,NaN,NaN,Premjit Lall,R,NaN,IND,27.1,100119,NaN,NaN,Ilie Nastase,R,183.0,ROU,21.4,6-3 2-6 6-4,3,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.000,0.000,1516.000,1484.000,0.0,0.0,0.0,0.0,1516.000,1484.000
3,3756,1968-T101,Bombay,clay,NaN,A,19671228,104,100035,NaN,NaN,Ion Tiriac,R,185.0,ROU,28.6,114613,NaN,NaN,Shyam Minotra,U,NaN,IND,21.2,6-4 6-4,3,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.000,0.000,1516.000,1484.000,0.0,0.0,0.0,0.0,1516.000,1484.000
4,3760,1968-T102,Calcutta,unknown,NaN,A,19671228,101,100035,NaN,NaN,Ion Tiriac,R,185.0,ROU,28.6,100141,NaN,NaN,Zeljko Franulovic,R,NaN,CRO,20.5,6-3 6-3 4-6 8-6,5,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,1531.264,1484.736
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192304,1382,2024-M-DC-2024-WG2-PO-BOL-THA-01,Davis Cup WG2 PO: BOL vs THA,clay,4.0,D,20240203,4,106262,NaN,NaN,Boris Arias,R,NaN,BOL,30.2,207788,NaN,NaN,Thanapet Chanta,U,NaN,THA,24.3,6-4 6-3,3,0,63.0,8.0,3.0,53.0,33.0,27.0,13.0,10.0,3.0,3.0,4.0,7.0,70.0,38.0,28.0,13.0,9.0,6.0,9.0,NaN,NaN,938.0,16.0,0.0,0.0,0.000,0.000,1516.000,1484.000,0.0,0.0,0.0,0.0,1516.000,1484.000
192310,1395,2024-M-DC-2024-WG2-PO-HKG-ZIM-01,Davis Cup WG2 PO: HKG vs ZIM,hard,4.0,D,20240203,4,212721,NaN,NaN,Benedict Badza,U,NaN,ZIM,18.6,212722,NaN,NaN,Kwok Shun Dasson Chan,U,NaN,HKG,18.5,6-7(5) 6-4 [6-10],3,0,119.0,11.0,4.0,82.0,51.0,40.0,16.0,11.0,1.0,2.0,2.0,2.0,83.0,49.0,35.0,17.0,11.0,0.0,2.0,NaN,NaN,NaN,NaN,0.0,0.0,1516.000,1484.000,0.000,0.000,0.0,0.0,0.0,0.0,1516.000,1484.000
192314,1399,2024-M-DC-2024-WG2-PO-JAM-BAR-01,Davis Cup WG2 PO: JAM vs BAR,hard,4.0,D,20240203,5,209868,NaN,NaN,Kaipo Marshall,R,NaN,BAR,21.8,202120,NaN,NaN,Rowland Phillips,R,NaN,JAM,30.0,6-4 1-6 6-2,3,0,128.0,1.0,14.0,84.0,52.0,35.0,11.0,13.0,5.0,9.0,3.0,3.0,84.0,55.0,34.0,12.0,12.0,3.0,8.0,NaN,NaN,NaN,NaN,0.0,0.0,1457.635,1461.418,0.000,0.000,0.0,0.0,0.0,0.0,1462.231,1463.052
192315,1400,2024-M-DC-2024-WG2-PO-PAR-MON-01,Davis Cup WG2 PO: PAR vs MON,clay,4.0,D,20240203,1,118808,NaN,NaN,Daniel Vallejo,R,NaN,MEX,39.6,104112,NaN,NaN,Benjamin Balleret,R,185.0,MON,41.0,6-2 6-4,3,0,67.0,5.0,0.0,42.0,29.0,20.0,12.0,9.0,0.0,1.0,3.0,0.0,54.0,31.0,17.0,10.0,9.0,3.0,7.0,NaN,NaN,NaN,NaN,0.0,0.0,0.000,0.000,1518.706,1540.635,0

In [ ]:
all_rankings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Tennis Project/all_rankings.csv')

In [ ]:
all_rankings[(all_rankings['ranking_date'] <= 20240220) & (all_rankings['ranking_date'] >= 20240125)]

,Unnamed: 0,ranking_date,rank,player,points
3206779,6171,20240129,1,104925,9855.0
3206780,6172,20240129,2,207989,9255.0
3206781,6173,20240129,3,106421,8765.0
3206782,6174,20240129,4,206173,8310.0
3206783,6175,20240129,5,126094,5050.0
...,...,...,...,...,...
3214995,14387,20240219,2044,207982,1.0
3214996,14388,20240219,2049,131888,1.0
3214997,14389,20240219,2049,210649,1.0
3214998,14390,20240219,2049,211765,1.0
